## Import dependancies and set environment determinism

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
#     tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(
    0
)
import numpy as np
import random
import pandas as pd


SEED = 378
# SEED = 123
print(SEED)
def set_seeds(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

def set_global_determinism(seed=SEED):
    set_seeds(seed=seed)

    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

# Call the above function with seed value
set_global_determinism(seed=SEED)

    


In [ ]:
import glacierml as gl
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tqdm import tqdm
import path_manager as pm
[
        home_path, data_path, RGI_path, glathida_path, 
        ref_path, coregistration_testing_path, 
        arch_test_path, LOO_path
] = pm.set_paths()

In [ ]:
def run_model(model_path,l1,l2,loss = 'mae'):
            
    normalizer = preprocessing.Normalization(axis=-1)
    normalizer.adapt(np.array(trfeat))

    model = gl.build_dnn_model(
        normalizer, learning_rate = 0.01, 
        layer_1 = l1, layer_2 = l2,loss = loss
    )

    model_history = model.fit(
        trfeat,
        trlabs,
        validation_split=0.2,
        callbacks = [callback],
        verbose=0, 
        epochs=500
    )
    model_filename = os.path.join(model_path)
    model.save(model_filename)
    return model

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    min_delta = 0.001,
    patience = 10,
    verbose = 0,
    mode = 'auto',
    baseline = None,
    restore_best_weights = True
)

In [ ]:
for i in range(1,5,1):
#     l1 = 8
#     l2 = 5
    if i == 1:
        l1 = 10
        l2 = 5
        
    if i == 2:
        l1 = 8
        l2 = 2
        
    if i == 3:
        l1 = 9
        l2 = 5
        
    if i == 4:
        l1 = 6
        l2 = 2
    set_global_determinism(seed=SEED)
    df = gl.coregister_data(data_path,str(i))
    df = df.drop(df[df['RGIId'].duplicated(keep = False)].index)
    df = df.reset_index().drop('index', axis = 1)
    trfeat, tefeat, trlabs, telabs = gl.split_data(df)
    print(trfeat)
    

    model = {}
    model_history = {}
    normalizer = {}
    model_path = os.path.join(
        coregistration_testing_path, str(i)
    )
    model = run_model(model_path,l1,l2)
    
    rgi_est_pth = os.path.join(model_path, 'rgi_est_raw.pkl')

    if os.path.isdir(rgi_est_pth) == False:

        RGI = gl.load_RGI(RGI_path)
#         RGI['Lmax'] = RGI['Lmax'] / 1e3
        rfp = RGI[list(df)[:-1]]

        preds = pd.Series(
            model.predict(rfp.drop('RGIId',axis = 1)).flatten(), name = i
        )
        RGI = pd.concat([RGI,preds], axis = 1)
        RGI.to_pickle(rgi_est_pth)
        print(f'Coregistration {i} predicted RGI')
                # RGI = pd.read_pickle('rgi_est_raw.pkl')